Program for reconstructing holograms from a config file generated with $\mathtt{FromMatlab.ipynb}$. You can still change the beamstop size, the ROI and the propagation distance.

# Load libraries

Import the libraries. Execute this cell only once when the kernel is started.

In [1]:
# import self written libraries. These have to be in the same folder or the parent directory.
import sys, os
sys.path.append('./library/')
import reconstruct as rec
import fth_reconstruction as fth
import pymaxp04 as maxi

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

#interactive stuff
import ipywidgets as widgets
from IPython.display import display
import ipython_blocking

In [2]:
%matplotlib widget

# Load images

Input the image numbers, first the positive helicity image followed by the negative helicity image. If you want to do a single helicity reconstruction, then put $\mathtt{np.nan}$ into the spot, where no image is. Please specify which config file you want to use.

You must also specify whether you want to use an autofactor when substracting the images (or the topography when doing a single helicity reconstruction -- recomended!) and which parameters you want to change (propagation, ROI, beamstop). Just set the according values to True/False.

Input the file folders for the data, the matlab files and where the results are saved (check, if this is correct, especially when switching from Linux to Windows and vice versa!).

**You can execute the integrity of the following code with -> "Cell" -> "Run Cells Below". Some interaction is needed and the code will stop there and resume when you are finished.**

In [3]:
experimental_setup = {'ccd_dist': 18e-2, 'energy': 779.5, 'px_size' : 20e-6}

In [4]:
images = np.array([148])
topo = np.array([148, 148])
conf_nr = 148

fname = 'test_%05d.nxs'

autoFactor = True
propagate = False
ROI = False
beamstop = False

In [5]:
folder = '/home/mschndr/MaxP04_2103_commissioning/raw/'
conf_folder = './processed/%s/' %conf_nr

In [6]:
INSTRUMENT_KEYS = {
    'mono': 'scan/instrument/mono/energy',
    'undu_shift': 'scan/instrument/undulator/collection/shift',
    'ringcurrent': 'scan/instrument/petra_iii/current',
    'exitslit': 'scan/instrument/exit_slit/slit/y_gap'
    }

Determine whether a double helicity or single helicity (positive or negative) is done

Load the images, add them up, shift the center.

Returns the shifted hologram, the center coordinates the beamstop diameter and the propagation distance from the config file.

In [7]:
# topo_data= maxi.get_mte(folder + '%s.h5'%fname, topo[0]) + maxi.get_mte(folder + '%s.h5'%fname, topo[1])

# # Modified for Sum
# if np.isnan(images[0]):
#     im = int(images[1])
#     neg = np.zeros((1300, 1340))
#     for i in range(im, im+100):
#         neg += maxi.get_mte(folder + '%s.h5'%fname, i)
#     holo, factor, center, bs_diam, roi, prop_dist, phase = rec.fromParameters(None, neg, conf_folder + 'P04_0620_%s-%d.hdf'%(fname,conf_nr),
#                                                                               new_bs=beamstop, old_prop= not(propagate),
#                                                                               topo= topo_data, auto_factor=autoFactor)
# elif np.isnan(images[1]):
#     im = int(images[0])
#     pos = np.zeros((1300, 1340))
#     for i in range(im, im+100):
#         pos += maxi.get_mte(folder + '%s.h5'%fname, i)
#     holo, factor, center, bs_diam, roi, prop_dist, phase = rec.fromParameters(pos, None, conf_folder + 'P04_0620_%s-%d.hdf'%(fname,conf_nr),
#                                                                               new_bs=beamstop, old_prop= not(propagate),
#                                                                               topo= topo_data, auto_factor=autoFactor)
# else:
#     neg = np.zeros((1300, 1340))
#     for i in range(int(images[1]), int(images[1])+1):
#         neg += maxi.get_mte(folder + '%s.h5'%fname, i)
#     pos = np.zeros((1300, 1340))
#     for i in range(int(images[0]), int(images[0])+1):
#         pos += maxi.get_mte(folder + '%s.h5'%fname, i)
#     im = images[0]
#     holo, factor, center, bs_diam, roi, prop_dist, phase = rec.fromParameters(pos, neg, conf_folder + 'P04_0620_%s-%d.hdf'%(fname,conf_nr),
#                                                                               new_bs=beamstop, old_prop= not(propagate),
#                                                                               topo= None, auto_factor=autoFactor)

In [8]:
np.array([13])[0].astype(int)

13

In [30]:
if len(images) == 1:
    im = images[0].astype(np.int)
    neg = maxi.load_scanfile(folder + fname%im, INSTRUMENT_KEYS)
    if neg.undu_shift > 0:
        pos = deepcopy(neg)
        neg = None
        hel = True
    else:
        pos = None
        hel = False
    tp = maxi.load_scanfile(folder + fname%topo[0], INSTRUMENT_KEYS) + maxi.load_scanfile(folder + fname%topo[1], INSTRUMENT_KEYS)
else:
    image = images.astype(np.int)
    tmp = maxi.load_scanfile(folder + fname%image[0], INSTRUMENT_KEYS)
    if tmp.undu_shift > 0:
        pos = deepcopy(tmp)
        neg = maxi.load_scanfile(folder + fname%image[1], INSTRUMENT_KEYS)
    else:
        neg = deepcopy(tmp)
        pos = maxi.load_scanfile(folder + fname%image[1], INSTRUMENT_KEYS)
    hel = None
    im = image[0]
    topo=None

In [37]:
pos=np.array(pos["ccd"][0])
neg=np.array(neg["ccd"][0])

In [57]:
try:    
    print(conf_folder + 'P04_0620_%04d.hdf'%(conf_nr))
    pos, neg, intercept, slope, center, bs_diam, roi_array, prop_dist, phase, dx, dy = rec.fromParameters(pos, neg, conf_folder + 'P04_0620_%04d.hdf'%(conf_nr),
                                                                              new_bs=beamstop, old_prop= not(propagate),
                                                                              topo = tp, auto_factor=autoFactor,experimental_setup = experimental_setup)
    roi=np.s_[roi_array[2]:roi_array[3],roi_array[0]:roi_array[1]]
    holo=pos-neg
except:
    print('you were too eager! Try again ;)')

./processed/148/P04_0620_0148.hdf
neg=0.000 + 1.000*pos
Start reconstructing the image using the center and beamstop mask from the Matlab reconstruction.
Shifted image by -27 pixels in x and -39 pixels in y.
Shifted image by -27 pixels in x and -39 pixels in y.
Using beamstop diameter 33 from config file and a sigma of 10.
Using propagation distance from config file.
Now determine the global phase shift by executing phase_shift.


In [58]:
holo=np.abs(holo)

In [59]:
fig, ax = plt.subplots()
ax.imshow(holo)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cosmic Rays

In [60]:
# fig, ax = plt.subplots(figsize = (8, 8))
# ax.imshow(np.abs(holo)>1000)

In [61]:
# holo = remove_two(holo, [1126, 1127], 1888)

# holo = fth.remove_cosmic_ray(holo, [290, 642])

In [62]:
# holo = np.delete(holo, 68, axis = 1)
# holo = np.delete(holo, 0, axis = 0)

In [63]:
# holo[1851, 1941:1949] = holo[1852, 1941:1949]
# holo[1850, 1951:1960] = holo[1849, 1951:1960]

# Beamstop

You may change the beamstop diameter in the text widget below the following cell. The default value in the field is the beamstop diameter from the config file.

Click on the button when you are finished to resume the execution of the code. 

In [64]:
if beamstop:
    bs_input, bs_button = rec.change_beamstop(bs_diam)

In [65]:
if beamstop:    
    %block bs_button

In [66]:
if beamstop:
    bs_diam = bs_input.value
    holo = rec.set_beamstop(holo, bs_diam)
    pos = rec.set_beamstop(pos, bs_diam)
    neg = rec.set_beamstop(neg, bs_diam)

In [67]:
if np.logical_and(beamstop, not propagate):
    
    holo = fth.propagate(holo, prop_dist*1e-6)
    pos = fth.propagate(pos, prop_dist*1e-6)
    neg = fth.propagate(neg, prop_dist*1e-6)

# ROI

You may select the reconstruction you want to look at. Do this by zooming into the image and putting in the x and y coordinates of the start and end points of your ROI.

Click on the button when you are finished to resume the execution of the code. 

In [68]:
if ROI:
    fig, ax = fth.plot(np.real(fth.reconstruct(holo)), colorbar = False, scale = (2,98))

In [69]:
if ROI:
    x1, x2 = ax.get_xlim()
    y2, y1 = ax.get_ylim()
    roi_array = np.array([fth.integer(x1), fth.integer(x2), fth.integer(y1), fth.integer(y2)]) #xstart, xstop, ystart, ystop
    roi = np.s_[fth.integer(y1) : fth.integer(y2), fth.integer(x1) : fth.integer(x2) ]

# Propagate and Shift Phase

Now, the hologram can to be propagated. Choose a mode for putting the hologram together:
* "-": pos - neg
* "+": pos + neg
* "/": pos / neg
* "-/+": (pos - neg) / (pos + neg)

Use the slider "propagation" and chose a value for which the image is sharp.

Then you need to apply a global phase shift to the image, to get all the information into one part of the complex image (usually the real part).

Click on the button when you are finished to resume the execution of the code. 

In [73]:
mode="+"
slider_prop, slider_phase, slider_dx, slider_dy = rec.focus(pos, neg, roi, phase=phase, prop_dist=prop_dist,dx=dx, dy=dy, experimental_setup=experimental_setup, operation=mode, max_prop_dist=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='propagation[um]', layout=Layout(width='90%'), max=0.…

In [74]:
%block button

KeyError: 'button'

In [75]:
prop_dist=slider_prop.value
phase = slider_phase.value
dx=slider_dx.value
dy=slider_dy.value

# Save

Save the reconstruction (only the selected ROI) as a numpy array and all the parameters in a config file.

In [76]:
folder_save = '../../processed/%s/'%fname

if not(os.path.exists(folder_save)):
    print("Creating folder " + folder_save)
    os.mkdir(folder_save)

Creating folder ../../processed/test_%05d.nxs/


FileNotFoundError: [Errno 2] No such file or directory: '../../processed/test_%05d.nxs/'

In [77]:
recon = fth.reconstruct(fth.propagate(pos-neg, prop_dist*1e-6, experimental_setup = experimental_setup)*np.exp(1j*phase))
recon = fth.sub_pixel_centering(recon, dx, dy)[roi]

In [78]:
fig, ax = plt.subplots(frameon = False, figsize = (recon.shape[1] / 40, recon.shape[0] / 40))
ax.imshow(np.real(recon), cmap = 'gray')
ax.set_axis_off()
ax.annotate('%s-%d'%(fname, im), (.02, .95), xycoords = 'axes fraction', bbox = {'alpha': .5, 'ec': None, 'fc': 'w', 'lw': None})
plt.savefig(folder_save + 'P04_0620_%s-%d.png'%(fname,im), bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FileNotFoundError: [Errno 2] No such file or directory: '../../processed/test_%05d.nxs/P04_0620_test_%05d.nxs-148.png'

In [79]:
rec.save_parameters(folder_save + 'P04_0620_%s-%d.hdf'%(fname,im), recon, intercept, slope, center, bs_diam, prop_dist, phase, dx, dy, roi, images,
                     comment = 'reconstruction KG', topo = topo)

OSError: Unable to create file (unable to open file: name = '../../processed/test_%05d.nxs/P04_0620_test_%05d.nxs-148.hdf', errno = 2, error message = 'No such file or directory', flags = 15, o_flags = c2)